In [ ]:
import pandas as pd
import pandas
import subprocess

In [ ]:
url_index = pandas.read_csv("single_ends.html", header = None, skiprows=9, delimiter=' ')

In [ ]:
raw_list = list(url_index[16])
url_list = []
for url in raw_list:
  if (isinstance(url, str)):
    url_list.append(url.split('"')[1])
print(len(url_list))

18


In [ ]:
for url in url_list[0:1]:
    if url == '\n':
        continue
    wget_string = "wget " + url #x[1]
    subprocess.call(wget_string, shell=True)
    print(wget_string)
subprocess.call('gunzip *.gz', shell=True)

wget ftp://bioinfo.bti.cornell.edu:21/pub/other/ForAnna/single_end/L1-1.fastq.gz


0

In [ ]:
import glob, os
os.chdir(".")
fastq_files = []
for file in glob.glob("*.fastq"):
    fastq_files.append(file)
fastq_files

['L1-1.fastq']

In [ ]:
#trimmomatic
prinseq_names = [] 
for file in fastq_files:
  fqc = "java -jar ../Trimmomatic-0.39/trimmomatic-0.39.jar SE " + file + " " + file[:-6] + '_out.fastq -threads 6 ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:1:TRUE SLIDINGWINDOW:4:20 LEADING:3 TRAILING:3 MINLEN:40'
  prinseq_names.append(file[:-6] + '_out.fastq')
  print(fqc)
  subprocess.call(fqc, shell=True)

['L1-1_out.fastq']

In [ ]:
#prinseq
bowtie_names = []
for name in prinseq_names:
  prin = "prinseq++ -threads 6 -VERBOSE 0 -fastq " + name + " -min_len 40 -trim_tail_left 10 -trim_tail_right 10 -out_name " + name[:-10]
  bowtie_names.append(name[:-10] + '_good_out.fastq')
  print(prin)
  #subprocess.call(prin, shell=True)
bowtie_names

prinseq++ -threads 6 -VERBOSE 0 -fastq L1-1_out.fastq -min_len 40 -trim_tail_left 10 -trim_tail_right 10 -out_name L1-1


['L1-1_good_out.fastq']

Auti clean 1 goes up until prinseq


In [ ]:
import pandas as pd
import pandas
import subprocess

In [ ]:
import glob, os
os.chdir(".")
fastq_good = []
for file in glob.glob("prinseq/*good*.fastq"):
    fastq_good.append(file)
fastq_good

In [ ]:
#bowtie
for name in fastq_good[0:1]:
  bow = "bowtie -v 3 -k 1 -p 6 -x rRNA_database/rRNA_database.fa " + name + " bowtie/" + name[8:-15] + ".sam"
  print(bow)
  subprocess.call(bow, shell=True)

In [ ]:
#hisat
for name in fastq_good[0:1]:
  hisat = "hisat2 --rna-strandness R -p 6 --summary-file hisat_summary/" + name[8:-15] + ".txt  -x /home/tdk38/REU/RNA-Seq/auto_clean/hisat_index/index -k 20 -U " + name + " -S hisat_output/" + name[8:-15] + ".sam" 
  print(hisat)
  subprocess.call(hisat, shell=True)

In [ ]:
from Bio import SeqIO
import sys

ffile = SeqIO.parse("L11-1_good_out", "fastq")
header_set = set(line.strip() for line in open(L11-1.sam))

for seq_record in ffile:
    try:
        header_set.remove(seq_record.name)
    except KeyError:
        print(seq_record.format("fastq"))
        continue
if len(header_set) != 0:
    print(len(header_set),'of the headers from list were not identified in the input fasta file.', file=sys.stderr)